<a href="https://colab.research.google.com/github/adga195/Solucion_Reto_SC_63_JorgeAdrianGaetaMartinez/blob/main/Solucion_Reto_SC_63_JorgeAdrianGaetaMartinez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aprendizaje supervisado

Jorge Adrián Gaeta Martínez

**1. Carga de librerías**

In [75]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import seaborn as sns

**2. Carga de datos**

In [76]:
data = pd.read_csv("./bank_marketing_RETO_DS_AS.csv")
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,31,self-employed,married,tertiary,no,2666,no,no,cellular,10,nov,318,2,97,6,success,yes
1,29,unemployed,single,unknown,no,1584,no,no,cellular,6,sep,245,1,-1,0,unknown,yes
2,41,blue-collar,married,secondary,no,2152,yes,no,cellular,17,nov,369,1,-1,0,unknown,no
3,50,blue-collar,married,secondary,no,84,yes,no,cellular,17,jul,18,8,-1,0,unknown,no
4,40,admin.,married,secondary,no,0,no,no,cellular,28,jul,496,2,182,11,success,yes


**3. Obtener la información de la base de datos**

In [77]:
data.shape

(9000, 17)

In [78]:
data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [79]:
data.dtypes

,0
age,int64
job,object
marital,object
education,object
default,object
balance,int64
housing,object
loan,object
contact,object
day,int64


In [80]:
data.isnull().sum()

,0
age,0
job,0
marital,0
education,0
default,0
balance,0
housing,0
loan,0
contact,0
day,0


**Justificación y descripción de las variables**

a. Variables numéricas continuas: **age, balance, duration, campaign, pdays, previous** - Estas variables están tomando valores que son de rango amplio y no se pueden incluir en una categoría específica.

b. Variables númericas discretas: **day** - Es númerica pero la diferencia con las anteriores es que sólo toma valores limitados a los días de los meses, del 1 hasta el 31 en ciertos meses.

c. Variables categóricas nominales: **job, marital, education, concat, month, poutcome** - Para estas variables no hay un orden concreto, simplemente son agrupaciones sin jerarquía.

d. Variables categóricas binarias: **default, housing, loan** - Estas variables sólo tienen dos valores posibles que son "yes" y "no".

**4. Procesamiento de las variables**

In [81]:
numeric_vars = data.select_dtypes(include=['int64','float64']).columns
categorical_vars = data.select_dtypes(include=['object']).columns

**4a. Transformación de variables categóricas (6.a)**

Para esta transformación estoy usando OneHotEncoder con get_dummies porque las variables categóricas que tenemos en el set de datos son nominales y no tienen un orden concreto, con esta transformación se evitan las relaciones numéricas innecesarias que afectan a los algoritmos como regresión logística y redes neuronales.

Las variables numéricas se estandarizan con StandardScaler para normalizar la escala y hacer que el entrenamiento sea estable, cómo mencioné anteriormente, los algoritmos de este tipo son sensibles a las magnitudes de las variables y el algoritmo converge más rápido.




In [82]:
data_encoded = pd.get_dummies(data, columns=categorical_vars, drop_first=True)

**4b. Transformación de variables numéricas**

In [83]:
scaler = StandardScaler()
data_encoded[numeric_vars] = scaler.fit_transform(data_encoded[numeric_vars])

**5. Partición en entrenamiento, validación y prueba**

In [84]:
X = data_encoded.drop("y_yes", axis=1)
y = data_encoded["y_yes"]

# 60% Entrenamiento, 40% (20% Validación, 20% Pruebas)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.40, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

**6. Implementación de modelos**

In [85]:
# Regresión Logística

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [86]:
# Validación de la Regresión Logística

y_pred_val_log = log_model.predict(X_val)
y_pred_val_log = log_model.predict(X_val)

mc_log = confusion_matrix(y_val, y_pred_val_log)
cr_log = classification_report(y_val, y_pred_val_log, output_dict=False)

mc_df = pd.DataFrame(mc_log,
                     index=["Real_No", "Real_Sí"],
                     columns=["Pred_No", "Pred_Sí"])

print("=== Matriz de confusión ===\n")
print(mc_df)

print("\n=== Reporte de clasificación ===\n")
print(cr_log)


=== Matriz de confusión ===

         Pred_No  Pred_Sí
Real_No      928      130
Real_Sí      183      559

=== Reporte de clasificación ===

              precision    recall  f1-score   support

       False       0.84      0.88      0.86      1058
        True       0.81      0.75      0.78       742

    accuracy                           0.83      1800
   macro avg       0.82      0.82      0.82      1800
weighted avg       0.83      0.83      0.83      1800



In [87]:
# Entrenamiento de la red neuronal

mlp = MLPClassifier(hidden_layer_sizes=(16,8),
                    activation='relu',
                    solver='adam',
                    max_iter=500,
                    random_state=42)

mlp.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(16, 8), max_iter=500, random_state=42)

De la ejecución anterior se muestra el warning de que el modelo no converge despues de 500 iteraciones, y la configuración de la red neuronal.

Por ello vamos a utilizar una configuración diferente con más iteraciones e hiperparámetros distintos.

In [88]:
mlp = MLPClassifier(hidden_layer_sizes=(10,5),
                    activation='relu',
                    solver='adam',
                    learning_rate_init=0.001,
                    early_stopping=True,
                    n_iter_no_change=10,
                    max_iter=1500,
                    random_state=42)

mlp.fit(X_train, y_train)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(10, 5), max_iter=1500,
              random_state=42)

Con la nueva configuración ya no tenemos el warning pero el que la configuración tenga early stopping no garantiza que tengamos una convergencia como queremos, entonces se tiene que verificar si las iteraciones que el modelo ocupó son menores a la máxima cantidad de iteraciones que especificamos, que en este caso deberían ser 1,500.

In [89]:
mlp = MLPClassifier(hidden_layer_sizes=(10,5),
                    activation='relu',
                    solver='adam',
                    learning_rate_init=0.001,
                    n_iter_no_change=10,
                    max_iter=1500,
                    random_state=42)

mlp.fit(X_train, y_train)

print("Iteraciones:", mlp.n_iter_)
print("Max Iteraciones:", mlp.max_iter)

Iteraciones: 435
Max Iteraciones: 1500


In [90]:
# Validación de la red neuronal

y_pred_val_mlp = mlp.predict(X_val)

mc_mlp = confusion_matrix(y_val, y_pred_val_mlp)
mc_mlp_df = pd.DataFrame(mc_mlp,
                         index=["Real_No", "Real_Sí"],
                         columns=["Pred_No", "Pred_Sí"])

print("=== Matriz de confusión ===\n")
print(mc_mlp_df)

print("\n=== Reporte de clasificación ===\n")
print(classification_report(y_val, y_pred_val_mlp))


=== Matriz de confusión ===

         Pred_No  Pred_Sí
Real_No      901      157
Real_Sí      139      603

=== Reporte de clasificación ===

              precision    recall  f1-score   support

       False       0.87      0.85      0.86      1058
        True       0.79      0.81      0.80       742

    accuracy                           0.84      1800
   macro avg       0.83      0.83      0.83      1800
weighted avg       0.84      0.84      0.84      1800



**7. Evaluación final de los modelos**

In [91]:
best_model = mlp
y_pred_test = best_model.predict(X_test)

mc_test = confusion_matrix(y_test, y_pred_test)
mc_test_df = pd.DataFrame(mc_test,
                          index=["Real_No", "Real_Sí"],
                          columns=["Pred_No", "Pred_Sí"])

print("=== Mejor Modelo - Matriz Confusión (Test) ===\n")
print(mc_test_df)

print("\n=== Mejor Modelo — Reporte de clasificacion (Test) ===\n")
print(classification_report(y_test, y_pred_test))


=== Mejor Modelo - Matriz Confusión (Test) ===

         Pred_No  Pred_Sí
Real_No      839      166
Real_Sí      132      663

=== Mejor Modelo — Reporte de clasificacion (Test) ===

              precision    recall  f1-score   support

       False       0.86      0.83      0.85      1005
        True       0.80      0.83      0.82       795

    accuracy                           0.83      1800
   macro avg       0.83      0.83      0.83      1800
weighted avg       0.84      0.83      0.83      1800



In [92]:
best_model = log_model
y_pred_test = best_model.predict(X_test)

mc_test = confusion_matrix(y_test, y_pred_test)
mc_test_df = pd.DataFrame(mc_test,
                          index=["Real_No", "Real_Sí"],
                          columns=["Pred_No", "Pred_Sí"])

print("=== Mejor Modelo - Matriz Confusión (Test) ===\n")
print(mc_test_df)

print("\n=== Mejor Modelo — Reporte de clasificacion (Test) ===\n")
print(classification_report(y_test, y_pred_test))


=== Mejor Modelo - Matriz Confusión (Test) ===

         Pred_No  Pred_Sí
Real_No      875      130
Real_Sí      193      602

=== Mejor Modelo — Reporte de clasificacion (Test) ===

              precision    recall  f1-score   support

       False       0.82      0.87      0.84      1005
        True       0.82      0.76      0.79       795

    accuracy                           0.82      1800
   macro avg       0.82      0.81      0.82      1800
weighted avg       0.82      0.82      0.82      1800



Después de evaluar ambos modelos, determiné que el MLP es el mejor. Aunque la regresión logística es buena, el MLP tiene mayor precisión (0.83 vs 0.82) y un mejor f1-score para la clase positiva (0.82 vs 0.79), que en este problema representa a los clientes que sí aceptan el producto.

Además, el MLP reduce el número de falsos negativos, lo cual es mejor en este problema que trata de identificar a los clientes con mayor probabilidad de aceptar un producto, por ello seleccionamos mlp como mejor modelo.


Finalmente vamos a determinar si el modelo esta bien ajustado, sobreentrenado levemente o demasiado ajustado.

En este caso estoy usando 3% y 10% como límites porque son rangos que normalmente funcionan bien para darse una idea de si el modelo está aprendiendo bien, si es estable o si esta sobreentrenado, que es cuando hay demasiada diferencia entre los valores de validación y los de entrenamiento.

In [93]:
y_pred_train_mlp = mlp.predict(X_train)
y_pred_val_mlp = mlp.predict(X_val)

print("=== RED NEURONAL (MLP) ===\n")

print("Precisión (Train):", (y_train == y_pred_train_mlp).mean())
print("Precisión (Val):", (y_val == y_pred_val_mlp).mean())

diff = abs((y_train == y_pred_train_mlp).mean() - (y_val == y_pred_val_mlp).mean())
print("\nDiferencia Train-Val:", diff)

if diff < 0.03:
    print("Modelo bien ajustado")
elif diff < 0.10:
    print("Sobreentrenamiento leve")
else:
    print("Demasiado ajustado")


=== RED NEURONAL (MLP) ===

Precisión (Train): 0.8866666666666667
Precisión (Val): 0.8355555555555556

Diferencia Train-Val: 0.05111111111111111
Sobreentrenamiento leve
